In [1]:
import os
import sys

def is_colab():
    return 'google.colab' in sys.modules

if is_colab():
    from google.colab import drive
    drive.mount('/content/drive')
    PROJECT_PATH = '/content/drive/MyDrive/NBS_Project'
else:
    # 自动根据当前脚本定位项目根路径
    PROJECT_PATH = os.path.abspath(os.path.join(os.path.dirname(__file__), '..')) \
        if '__file__' in globals() else os.getcwd()

if PROJECT_PATH not in sys.path:
    sys.path.append(PROJECT_PATH)

print(f"项目路径设置为: {PROJECT_PATH}")

项目路径设置为: c:\Users\a1831\Desktop\NBS_Project


In [2]:
from src.config import Config
from src.main import create_MDs,create_ESP
import src.baseline as baseline
import src.NBS as NBS
import src.model as model
import numpy as np

CONFIG_PATH = os.path.join(PROJECT_PATH, "configs", "test.json")
config = Config(CONFIG_PATH)

# Create MDs with a specific seed for reproducibility
seed = 41
MDs = create_MDs(config, seed=seed)
# np.random.shuffle(MDs)  # Shuffle the MDs

for md in MDs:
    print(md.param)

esp = create_ESP(config, seed=seed)
print(esp.param)

{'s': 0.1, 'l': 0.8, 'cn': 0.32332202513299674, 'Fn': 1.184383282690447, 'kn': 0.6732799162883472, 'omega_n': 1, 'Rn': 4.13472934445215}
{'s': 0.1, 'l': 0.8, 'cn': 0.6374404631644437, 'Fn': 1.7637226511189565, 'kn': 0.6651454029151388, 'omega_n': 1, 'Rn': 10.248845012378245}
{'s': 0.1, 'l': 0.8, 'cn': 0.395711266542491, 'Fn': 2.1321345406869066, 'kn': 0.19255662205025992, 'omega_n': 1, 'Rn': 6.716553285779903}
{'s': 0.1, 'l': 0.8, 'cn': 0.16187321635232887, 'Fn': 3.8199302863134226, 'kn': 0.3183833952999494, 'omega_n': 1, 'Rn': 9.231363952354403}
{'s': 0.1, 'l': 0.8, 'cn': 0.6413215489755134, 'Fn': 3.913825949541608, 'kn': 0.4233228768151258, 'omega_n': 1, 'Rn': 7.948101841125762}
{'s': 0.1, 'l': 0.8, 'cn': 0.49308149762923636, 'Fn': 2.4920855793255217, 'kn': 0.5819339189335254, 'omega_n': 1, 'Rn': 3.859406010253179}
{'s': 0.1, 'l': 0.8, 'cn': 0.17404615893711323, 'Fn': 1.5048957788256199, 'kn': 0.3264336993988291, 'omega_n': 1, 'Rn': 7.130152926250668}
{'s': 0.1, 'l': 0.8, 'cn': 0.350

In [3]:
T = 100
N = len(MDs)
lam_uni, p_uni, r_uni, Dmax_uni = np.zeros(N),np.zeros(N),np.zeros(N),0
for _ in range(T):
    l,p,r,d = baseline.uniform_baseline(esp, MDs,43)
    lam_uni += l
    p_uni += p 
    r_uni += r
    Dmax_uni += d
lam_uni, p_uni, r_uni, Dmax_uni = lam_uni/T, p_uni/T, r_uni/T, Dmax_uni/T

print(f"Uniform λ: {lam_uni}")
print(f"Uniform p: {p_uni}")  
print(f"Uniform Dmax: {Dmax_uni}")
print(f"Uniform r: {r_uni}")
print("==============================")
print("social welfare:",model.social_welfare(esp, MDs, lam_uni, p_uni))
print("sum lambda:", np.sum(lam_uni), " lambda0:", esp.lambda0)
print([f"MD {i+1} Dn: {md.delay(p_uni[i],lam_uni[i])}" for i, md in enumerate(MDs)])
print("Dmax:", Dmax_uni, " D0:", esp.D0)
max_Dn = max(md.delay(p_uni[i], lam_uni[i]) for i, md in enumerate(MDs))
print("max Dn:", max_Dn)
print("Dmax == max Dn:", Dmax_uni == max_Dn)
print("Dn<=Dmax:", all(md.delay(p_uni[i], lam_uni[i]) <= Dmax_uni for i, md in enumerate(MDs)))
print("Dmax<=D0:", Dmax_uni <= esp.D0)
print("pn<=Fn:", all(p_uni[i] <= md.Fn for i, md in enumerate(MDs)))
print("lambda_n<=pn/sl:", all(lam_uni[i] <= p_uni[i] / (md.s*md.l) for i, md in enumerate(MDs)))
print("sum r:", np.sum(r_uni), "Q_star:", esp.Q(max_Dn))
print("rn>=Ln:", all(r_uni[i] >= md.Ln(p_uni[i]) for i, md in enumerate(MDs)))
print("sum Ln:", np.sum([md.Ln(p_uni[i]) for i,md in enumerate(MDs)]))

Uniform λ: [5. 5. 5. 5. 5. 5. 5. 5. 5. 5.]
Uniform p: [0.92100001 0.48527993 0.46095682 3.27435173 2.47417436 0.89095722
 1.23730207 1.19202808 1.87957382 2.17605924]
Uniform Dmax: 1.3272929506848081
Uniform r: [23.04668976 23.04668976 23.04668976 23.04668976 23.04668976 23.04668976
 23.04668976 23.04668976 23.04668976 23.04668976]
social welfare: 230.4668976068324
sum lambda: 50.0  lambda0: 50
['MD 1 Dn: 0.17773623972334668', 'MD 2 Dn: 0.9478442742383153', 'MD 3 Dn: 1.3272929506848183', 'MD 4 Dn: 0.03866499814551409', 'MD 5 Dn: 0.05115118397187841', 'MD 6 Dn: 0.18885771097720916', 'MD 7 Dn: 0.10956991337392175', 'MD 8 Dn: 0.11098137321648126', 'MD 9 Dn: 0.06614255302124675', 'MD 10 Dn: 0.06251306644019254']
Dmax: 1.3272929506848081  D0: 3
max Dn: 1.3272929506848183
Dmax == max Dn: False
Dn<=Dmax: False
Dmax<=D0: True
pn<=Fn: True
lambda_n<=pn/sl: True
sum r: 230.4668976068324 Q_star: 247.01083342594447
rn>=Ln: True
sum Ln: 16.543935819112093


In [4]:
lam_pro, p_pro, r_pro, Dmax_pro = baseline.proportional_baseline(esp,MDs)
print(f"Uniform λ: {lam_pro}")
print(f"Uniform p: {p_pro}")  
print(f"Uniform Dmax: {Dmax_pro}")
print(f"Uniform r: {r_pro}")
print("==============================")
print("social welfare:",model.social_welfare(esp, MDs, lam_pro, p_pro))
print("sum lambda:", np.sum(lam_pro), " lambda0:", esp.lambda0)
print([f"MD {i+1} Dn: {md.delay(p_pro[i],lam_pro[i])}" for i, md in enumerate(MDs)])
print("Dmax:", Dmax_pro, " D0:", esp.D0)
max_Dn = max(md.delay(p_pro[i], lam_pro[i]) for i, md in enumerate(MDs))
print("max Dn:", max_Dn)
print("Dmax == max Dn:", Dmax_pro == max_Dn)
print("Dn<=Dmax:", all(md.delay(p_pro[i], lam_pro[i]) <= Dmax_pro for i, md in enumerate(MDs)))
print("Dmax<=D0:", Dmax_pro <= esp.D0)
print("pn<=Fn:", all(p_pro[i] <= md.Fn for i, md in enumerate(MDs)))
print("lambda_n<=pn/sl:", all(lam_pro[i] <= p_pro[i] / (md.s*md.l) for i, md in enumerate(MDs)))
print("sum r:", np.sum(r_pro), "Q_star:", esp.Q(max_Dn))
print("rn>=Ln:", all(r_pro[i] >= md.Ln(p_pro[i]) for i, md in enumerate(MDs)))
print("sum Ln:", np.sum([md.Ln(p_pro[i]) for i,md in enumerate(MDs)]))

Uniform λ: [2.09025991 3.11270752 3.76289958 6.74160743 6.90731927 4.39815949
 2.65591668 5.8750097  6.85821515 7.59790528]
Uniform p: [1.18438328 1.76372265 2.13213454 3.81993029 3.91382595 2.49208558
 1.50489578 3.32889859 3.88600257 4.30512586]
Uniform Dmax: 0.1028355471996345
Uniform r: [ 8.10768535 12.07354791 14.59550826 26.14929921 26.79206114 17.05954993
 10.30175083 22.78794606 26.60159642 29.47070127]
social welfare: 193.93964638257935
sum lambda: 50.0  lambda0: 50
['MD 1 Dn: 0.1028355471996345', 'MD 2 Dn: 0.0625727251662071', 'MD 3 Dn: 0.05857811286095347', 'MD 4 Dn: 0.03521840591599758', 'MD 5 Dn: 0.03638235768232111', 'MD 6 Dn: 0.06328983391342033', 'MD 7 Dn: 0.07592421010504175', 'MD 8 Dn: 0.03795766576644862', 'MD 9 Dn: 0.03604407740719918', 'MD 10 Dn: 0.03910697341732085']
Dmax: 0.1028355471996345  D0: 3
max Dn: 0.1028355471996345
Dmax == max Dn: True
Dn<=Dmax: True
Dmax<=D0: True
pn<=Fn: True
lambda_n<=pn/sl: True
sum r: 193.93964638257935 Q_star: 248.274174600214
rn>=

In [5]:
lam_non, p_non, r_non, Dmax_non = baseline.non_cooperative_baseline(esp,MDs)
print(f"Non-cooperative λ: {lam_non}")
print(f"Non-cooperative p: {p_non}")  
print(f"Non-cooperative Dmax: {Dmax_non}")
print(f"Non-cooperative r: {r_non}")
print("==============================")
print("social welfare:",model.social_welfare(esp, MDs, lam_non, p_non))
print("sum lambda:", np.sum(lam_non), " lambda0:", esp.lambda0)
print([f"MD {i+1} Dn: {md.delay(p_non[i],lam_non[i])}" for i, md in enumerate(MDs)])
print("Dmax:", Dmax_non, " D0:", esp.D0)
max_Dn = max(md.delay(p_non[i], lam_non[i]) for i, md in enumerate(MDs))
print("max Dn:", max_Dn)
print("Dmax == max Dn:", Dmax_non == max_Dn)
print("Dn<=Dmax:", all(md.delay(p_non[i], lam_non[i]) <= Dmax_non for i, md in enumerate(MDs)))
print("Dmax<=D0:", Dmax_non <= esp.D0)
print("pn<=Fn:", all(p_non[i] <= md.Fn for i, md in enumerate(MDs)))
print("lambda_n<=pn/sl:", all(lam_non[i] <= p_non[i] / (md.s*md.l) for i, md in enumerate(MDs)))
print("sum r:", np.sum(r_non), "Q_star:", esp.Q(max_Dn))
print("rn>=Ln:", all(r_non[i] >= md.Ln(p_non[i]) for i, md in enumerate(MDs)))
print("sum Ln:", np.sum([md.Ln(p_non[i]) for i,md in enumerate(MDs)]))

c:\Users\a1831\anaconda3\Lib\site-packages\scipy\optimize\_hessian_update_strategy.py:182: UserWarning: delta_grad == 0.0. Check if the approximated function is linear. If the function is linear better results can be obtained by defining the Hessian as zero instead of using quasi-Newton approximations.
  warn('delta_grad == 0.0. Check if the approximated '
c:\Users\a1831\anaconda3\Lib\site-packages\scipy\optimize\_optimize.py:353: RuntimeWarning: Values in x were outside bounds during a minimize step, clipping to bounds
  warnings.warn("Values in x were outside bounds during a "


✅ 这个点严格满足所有约束（在容差范围内）。
Positive directional derivative for linesearch    (Exit mode 8)
            Current function value: -109.54479465295304
            Iterations: 55
            Function evaluations: 1196
            Gradient evaluations: 51
求解失败：8 : Positive directional derivative for linesearch
Non-cooperative λ: [1.47260054e+00 1.58756242e-12 8.81833529e+00 1.23563341e+01
 4.24129736e+00 3.53570074e+00 9.30581780e+00 9.96655932e-01
 8.12533780e+00 1.14958874e+00]
Non-cooperative p: [0.1450211695971126, 0, 0.7325941611477165, 1.015596807883097, 0.3664099230109353, 0.31008516690063026, 0.7715848201424343, 0.10681468820983485, 0.6771284868892068, 0.11911819899408618]
Non-cooperative Dmax: 2.9639431280720743
Non-cooperative r: [0.09147835 0.00771469 0.29379226 0.30508125 0.14949681 0.16460653
 0.33477097 0.07276379 0.28576432 0.07991473]
social welfare: 109.47324532045435
sum lambda: 50.001668296048436  lambda0: 50
['MD 1 Dn: 2.9639431280720765', 'MD 2 Dn: 0', 'MD 3 Dn: 2.9639431280

In [6]:
lam_swm, p_swm, Dmax_swm = baseline.social_welfare_maximization(esp, MDs)
print(f"SWM λ: {lam_swm}")
print(f"SWM p: {p_swm}")  
print(f"SWM Dmax: {Dmax_swm}")
r_swm = baseline.solve_r_NBP(esp, MDs, Dmax_swm, lam_swm, p_swm)
print(f"SWM r: {r_swm}")
print("==============================")
print("social welfare:",model.social_welfare(esp, MDs, lam_swm, p_swm))
print("sum lambda:", np.sum(lam_swm), " lambda0:", esp.lambda0)
print([f"MD {i+1} Dn: {md.delay(p_swm[i],lam_swm[i])}" for i, md in enumerate(MDs)])
print("Dmax:", Dmax_swm, " D0:", esp.D0)
max_Dn = max(md.delay(p_swm[i], lam_swm[i]) for i, md in enumerate(MDs))
print("max Dn:", max_Dn)
print("Dmax == max Dn:", Dmax_swm == max_Dn)
print("Dn<=Dmax:", all(md.delay(p_swm[i], lam_swm[i]) <= Dmax_swm for i, md in enumerate(MDs)))
print("Dmax<=D0:", Dmax_swm <= esp.D0)
print("pn<=Fn:", all(p_swm[i] <= md.Fn for i, md in enumerate(MDs)))
print("lambda_n<=pn/sl:", all(lam_swm[i] <= p_swm[i] / (md.s*md.l) for i, md in enumerate(MDs)))
print("sum r:", np.sum(r_swm), "Q_star:", esp.Q(max_Dn))
print("rn>=Ln:", all(r_swm[i] >= md.Ln(p_swm[i]) for i, md in enumerate(MDs)))
print("sum Ln:", np.sum([md.Ln(p_swm[i]) for i,md in enumerate(MDs)]))

✅ 这个点严格满足所有约束（在容差范围内）。


c:\Users\a1831\anaconda3\Lib\site-packages\scipy\optimize\_optimize.py:353: RuntimeWarning: Values in x were outside bounds during a minimize step, clipping to bounds
  warnings.warn("Values in x were outside bounds during a "


Optimization terminated successfully    (Exit mode 0)
            Current function value: -243.3480999047845
            Iterations: 316
            Function evaluations: 7249
            Gradient evaluations: 316
SWM λ: [3.07863735 2.9162451  6.54226158 8.40311345 5.48324198 3.02016642
 9.41095494 1.75177419 7.60059485 1.79301014]
SWM p: [0.34633578 0.33157126 0.62227595 0.77065072 0.53727315 0.34187444
 0.85166595 0.23843987 0.70659958 0.24265237]
SWM Dmax: 0.8238281718887738
Optimization terminated successfully    (Exit mode 0)
            Current function value: -37.589656695490305
            Iterations: 12
            Function evaluations: 133
            Gradient evaluations: 12
SWM r: [20.70045783 20.68758208 20.65645702 20.63105094 20.72193568 20.62645879
 20.82758921 20.62501875 20.74497368 20.62340831]
social welfare: 245.14588308480256
sum lambda: 49.99999999999999  lambda0: 50
['MD 1 Dn: 0.8238271719158742', 'MD 2 Dn: 0.8238271719106848', 'MD 3 Dn: 0.8238271719160407', 'MD

In [7]:
lam_opt, p_opt, r_opt, Dmax_opt = baseline.optimal_NBP(esp, MDs)
print(f"Optimal λ: {lam_opt}")
print(f"Optimal p: {p_opt}")  
print(f"Optimal r: {r_opt}")
print(f"Optimal Dmax: {Dmax_opt}")
print("==============================")
print("social welfare:",model.social_welfare(esp, MDs, lam_opt, p_opt))
print("sum lambda:", np.sum(lam_opt), " lambda0:", esp.lambda0)
print([f"MD {i+1} Dn: {md.delay(p_opt[i],lam_opt[i])}" for i, md in enumerate(MDs)])
print("Dmax:", Dmax_opt, " D0:", esp.D0)
print("max Dn:", max(md.delay(p_opt[i], lam_opt[i]) for i, md in enumerate(MDs)))
print("Dmax == max Dn:", Dmax_opt == max(md.delay(p_opt[i], lam_opt[i]) for i, md in enumerate(MDs)))
print("Dn<=Dmax:", all(md.delay(p_opt[i], lam_opt[i]) <= Dmax_opt for i, md in enumerate(MDs)))
print("Dmax<=D0:", Dmax_opt <= esp.D0)
print("pn<=Fn:", all(p_opt[i] <= md.Fn for i, md in enumerate(MDs)))
print("lambda_n<=pn/sl:", all(lam_opt[i] <= p_opt[i] / (md.s*md.l) for i, md in enumerate(MDs)))
print("sum r:", np.sum(r_opt), "Q_star:", esp.Q(Dmax_opt))
print("rn>=Ln:", all(r_opt[i] >= md.Ln(p_opt[i]) for i, md in enumerate(MDs)))
print("sum Ln:", np.sum([md.Ln(p_opt[i]) for i,md in enumerate(MDs)]))

✅ 这个点严格满足所有约束（在容差范围内）。
Optimization terminated successfully    (Exit mode 0)
            Current function value: -35.98812131829895
            Iterations: 59
            Function evaluations: 1889
            Gradient evaluations: 59
Optimal λ: [4.8404622  4.74770872 5.17328246 5.31423996 4.98064244 4.93263002
 5.19162106 4.82944036 5.14614309 4.8438297 ]
Optimal p: [0.48926574 0.48000194 0.5146958  0.52545955 0.49899226 0.49686417
 0.51605325 0.48656778 0.51216807 0.48866867]
Optimal r: [20.81112359 20.82058158 20.56062763 20.51114944 20.65560883 20.72438297
 20.58857432 20.84049204 20.58283629 20.83146823]
Optimal Dmax: 0.8082780146622826
social welfare: 244.7511248467392
sum lambda: 50.0  lambda0: 50
['MD 1 Dn: 0.8082780146962282', 'MD 2 Dn: 0.8082780146168773', 'MD 3 Dn: 0.8082780146737935', 'MD 4 Dn: 0.8082780146416569', 'MD 5 Dn: 0.8082780146600207', 'MD 6 Dn: 0.8082780146613974', 'MD 7 Dn: 0.8082780146635227', 'MD 8 Dn: 0.8082780146686164', 'MD 9 Dn: 0.8082780146749876', 'MD 10

In [8]:
lamb_admm, p_admm, Dmax_admm = NBS.ADMM(esp, MDs)
print(f"ADMM λ: {lamb_admm}")
print(f"ADMM p: {p_admm}")  
print(f"ADMM Dmax: {Dmax_admm}")
r_admm = NBS.negotiation(esp, MDs,lamb_admm, p_admm, Dmax_admm)
print(f"ADMM r: {r_admm}")
print("==============================")
print("social welfare:",model.social_welfare(esp, MDs, lamb_admm, p_admm))
print("sum lambda:", np.sum(lamb_admm), " lambda0:", esp.lambda0)
print([f"MD {i+1} Dn: {md.delay(p_admm[i],lamb_admm[i])}" for i, md in enumerate(MDs)])
print("Dmax:", Dmax_admm, " D0:", esp.D0)
print("max Dn:", max(md.delay(p_admm[i], lamb_admm[i]) for i, md in enumerate(MDs)))
print("Dmax == max Dn:", Dmax_admm == max(md.delay(p_admm[i], lamb_admm[i]) for i, md in enumerate(MDs)))
print("Dn<=Dmax:", all(md.delay(p_admm[i], lamb_admm[i]) <= Dmax_admm for i, md in enumerate(MDs)))
print("Dmax<=D0:", Dmax_admm <= esp.D0)
print("pn<=Fn:", all(p_admm[i] <= md.Fn for i, md in enumerate(MDs)))
print("lambda_n<=pn/sl:", all(lamb_admm[i] <= p_admm[i] / (md.s*md.l) for i, md in enumerate(MDs)))
print("sum r:", np.sum(r_admm), "Q_star:", esp.Q(Dmax_admm))
print("rn>=Ln:", all(r_admm[i] >= md.Ln(p_admm[i]) for i, md in enumerate(MDs)))
print("sum Ln:", np.sum([md.Ln(p_admm[i]) for i,md in enumerate(MDs)]))

ADMM λ: [3.74277301 3.28907538 6.2326925  7.75001852 4.79436762 4.43723994
 6.49827496 3.5573493  6.02975915 3.66844962]
ADMM p: [0.40233396 0.3641628  0.60031111 0.72117507 0.48494786 0.45812017
 0.62144609 0.38565264 0.58371375 0.39550668]
ADMM Dmax: 0.8015507543823365
ADMM r: [20.74323953 20.71016912 20.63165232 20.60061251 20.66553784 20.7108399
 20.66727128 20.75576444 20.64845105 20.75677026]
social welfare: 245.01215503392157
sum lambda: 50.00000000000001  lambda0: 50
['MD 1 Dn: 0.8015476757616706', 'MD 2 Dn: 0.8015481566284446', 'MD 3 Dn: 0.8015491130088014', 'MD 4 Dn: 0.8015528323331875', 'MD 5 Dn: 0.8015482891428154', 'MD 6 Dn: 0.8015481423592132', 'MD 7 Dn: 0.801549806681369', 'MD 8 Dn: 0.8015470112829537', 'MD 9 Dn: 0.8015490423693208', 'MD 10 Dn: 0.8015471495305757']
Dmax: 0.8015507543823365  D0: 3
max Dn: 0.8015528323331875
Dmax == max Dn: False
Dn<=Dmax: False
Dmax<=D0: True
pn<=Fn: True
lambda_n<=pn/sl: True
sum r: 206.89030824956703 Q_star: 247.72566957824162
rn>=Ln: T

In [9]:
print("Nash Product Opt:",model.nash_product_log(esp, MDs, lam_opt, p_opt, r_opt))
print("Nash Product SWM:",model.nash_product_log(esp, MDs, lam_swm, p_swm, r_swm))
print("Nash Product ADMM:",model.nash_product_log(esp, MDs, lamb_admm, p_admm, r_admm))
print("Nash Product Proportional:",model.nash_product_log(esp, MDs, lam_pro, p_pro, r_pro))
print("Nash Product Uniform:",model.nash_product_log(esp, MDs, lam_uni, p_uni, r_uni))
print("Nash Product Non-cooperative:",model.nash_product_log(esp, MDs, lam_non, p_non, r_non))
print("=============================")
print("SW Opt:",model.social_welfare(esp, MDs, lam_opt, p_opt))
print("SW SWM:",model.social_welfare(esp, MDs, lam_swm, p_swm))
print("SW ADMM:",model.social_welfare(esp, MDs, lamb_admm, p_admm))
print("SW Proportional:",model.social_welfare(esp, MDs, lam_pro, p_pro))
print("SW Uniform:",model.social_welfare(esp, MDs, lam_uni, p_uni))
print("SW Non-cooperative:",model.social_welfare(esp, MDs, lam_non, p_non))
print("=============================")
print("Dmax Opt:", Dmax_opt)
print("Dmax SWM:", Dmax_swm)
print("Dmax ADMM:", Dmax_admm)
print("Dmax Proportional:", Dmax_pro)
print("Dmax Uniform:", Dmax_uni)
print("Dmax Uniform:", Dmax_non)

Nash Product Opt: 37.570317091183014
Nash Product SWM: 37.58965625766686
Nash Product ADMM: 37.58310842403301
Nash Product Proportional: 33.65776480543405
Nash Product Uniform: 36.22236437448023
Nash Product Non-cooperative: nan
SW Opt: 244.7511248467392
SW SWM: 245.14588308480256
SW ADMM: 245.01215503392157
SW Proportional: 193.93964638257935
SW Uniform: 230.4668976068324
SW Non-cooperative: 245.6345981044234
Dmax Opt: 0.8082780146622826
Dmax SWM: 0.8238281718887738
Dmax ADMM: 0.8015507543823365
Dmax Proportional: 0.1028355471996345
Dmax Uniform: 1.3272929506848081
Dmax Uniform: 0.7787181204944444


c:\Users\a1831\Desktop\NBS_Project\src\model.py:12: RuntimeWarning: invalid value encountered in log
  ns = [md.omega_n*np.log(rn-md.Ln(pn)) for (md,rn,pn) in zip(MDs,r,p)]
